# Initiation à Hadoop et Map-Reduce

L'Objectif du TP: 
Initiation au framework hadoop et au patron MapReduce, utilisation de docker pour lancer un cluster hadoop.
Les consignes de ce notebook sont à réaliser dans votre cluster deployé avec docker, mais vous pouvez utiliser le notebook pour stocker les commandes que vous avez utilisé afin de réaliser une sorte de cheat sheet.

- Clonez le repo à l’adresse https://github.com/big-data-europe/docker-hadoop dans votre répertoire de travail.


- Démarrez votre image docker: \
Ouvrez votre terminal et placez-vous dans le dossier docker-hadoop fraîchement récupéré. \
Lancez la commande: \
	docker-compose up -d


- Connectez vous au namenode de votre cluster:\
Lancez la commande: \
    docker exec -it namenode bash


Refs: \
Apache Hadoop [http://hadoop.apache.org/] \
Docker [https://www.docker.com/]



- Placez vous dans le dossier utilisateur: \
cd ~


- Créer un répertoire TP, puis deux sous-répertoires code et data dans lesquels sauvegarder respectivement les codes des mappers et reducers, et les données sources et résultats.


- En local (en dehors de docker), avec la commande docker adequate envoyez le fichier ventes.txt fourni, dans le répertoire ~/TP/data du namenode.

### 1- Hadoop HDFS
   
##### Les commandes les plus utilisées dans Hadoop: 
hdfs dfs –ls             Afficher le contenu du répertoire racine

hdfs dfs –put file.txt   Upload un fichier dans hadoop (à partir du répertoire courant linux)

hdfs dfs –get file.txt   Download un fichier à partir de hadoop sur votre disque local

hdfs dfs –tail file.txt  Lire les dernières lignes du fichier

hdfs dfs –cat file.txt   Affiche tout le contenu du fichier

hdfs dfs –mv file.txt newfile.txt   Renommer le fichier

hdfs dfs –rm newfile.txt            Supprimer le fichier

hdfs dfs –mkdir myinput             Créer un répertoire

hdfs dfs –cat file.txt | less       Lire le fichier page par page

- Créez un répertoire dans HDFS, appelé data.


- Envoyez le fichier ventes.txt depuis le dossier ~/TP/data vers le répertoire data HDFS.


- Affichez le contenu du répertoire data HDFS et visualisez les dernières lignes du fichier.

In [ ]:
Dans Windows terminal:
docker ps
docker cp ventes.txt 5ba:/TP/data/

Dans docker terminal:
/ls
mkdir /TP
cd TP/
mkdir data
cd data/
pwd
ls
hdfs dfs -put ventes.txt ./TP/data

cat /TP/data/ventes.txt | /TP/code/mapper.py

### 2-  Map Reduce
Un Job Map-Reduce se compose principalement de deux types de programmes:

Mappers : permettent d’extraire les données nécessaires sous forme de clef/valeur, pour pouvoir ensuite appliquer le shuffle &sort. \
Reducers : prennent un ensemble de données triées , et effectuent le traitement nécessaire sur ces données (somme, moyenne,total...)

PS : Vous aurez besoin d'installer python dans votre namenode docker avec apt-get pour tester vos mapper et reducer.

In [ ]:
Install Python:
    apt-get update
    apt-get install python3
    apt-get upgrade
    
    apt install nano
    
    ls -hal # voir les droits pour les fichiers
    chmod +rwx mapper.py # ajouter les droit pour ecrire, executer et lire
    nano mapper.py # pour modifier le fichier
    
    
https://www.michael-noll.com/tutorials/writing-an-hadoop-mapreduce-program-in-python/

##### 2.1 )

Soit le dataset ventes.txt comportant 6 champs, séparés par des tabulations, de la forme suivante:

date temps magasin produit coût paiement

Le but est de déterminer le total des ventes par magasin.

- Créez un fichier mapper.py dans le dossier ~/TP/code , qui permet de: \
Séparer les différents champs par tabulation. \
Extraire les éléments à partir de ces champs, sous forme de clé/valeur (magasin,coût), Pour calculer les ventes par magasin.

In [ ]:
#mapper.py
#!/usr/bin/python3

import sys

for line in sys.stdin:
        date, temps, magasin, produit, cout, paiement = line.strip().split('\t')
        print('{0}\t{1}'.format(magasin, cout))

- Testez votre mapper sur les 50 premières lignes du fichier ventes.txt 

In [ ]:
cat /TP/data/ventes.txt | /TP/code/mapper.py

##### 2.2 )
Le Reducer permet de faire le traitement définit sur des entrées préalablement triées par Hadoop sur les couples (magasin,coût). \
Le Reducer fera la somme de tous les coûts pour un même magasin. 

- Créez un fichier reducer.py dans le dossier ~/TP/code.


In [ ]:
#reducer.py
#!/usr/bin/python3
import sys


ventes_dict = {}

for line in sys.stdin:
        magasin, cout = line.strip().split('\t')
        if magasin not in ventes_dict:
                ventes_dict[magasin] = float(cout)
        else:
                ventes_dict[magasin] += float(cout)

for key, value in ventes_dict.items():
        print('{0}\t{1}'.format(key, value))

- Testez votre Reducer sur les 50 premières lignes du fichier ventes.txt

In [ ]:
cat /TP/data/ventes.txt | /TP/code/mapper.py | /TP/code/reducer.py

##### 2.3)
Lancer un job entier sur Hadoop =>  faire appel au mapper puis au reducer sur une entrée volumineuse, et obtenir à la fin un résultat, directement sur HDFS.

- Exécutez un job hadoop sur le fichier ventes.txt en utilisant les fichiers mapper.py et reducer.py déjà implémentés. 
- Stocker le résultat dans un répertoire joboutput.


- Sauvegarder ensuite le fichier part-00000 dans le dossier ~/TP/data.

In [ ]:
hdfs dfs -mkdir -p TP
hdfs dfs -ls
hdfs dfs -mkdir -p TP/code
hdfs dfs -mkdir -p TP/data
hdfs dfs -ls TP

hdfs dfs -put /TP/code/mapper.py ./TP/code
hdfs dfs -put /TP/code/reducer.py ./TP/code

hdfs dfs -ls TP/code
hdfs dfs -chmod +rwx TP/code/mapper.py
hdfs dfs -chmod +rwx TP/code/reducer.py

hdfs dfs -cat ./TP/data/ventes.txt | /TP/code/mapper.py | /TP/code/reducer.py #faux

In [ ]:
# install python in all images
docker exec -it namenode bash
apt update && apt upgrade -y && apt install python3 -y

In [ ]:
/opt/hadoop-3.2.1/share/hadoop/tools/lib/hadoop-streaming-3.2.1.jar

hadoop jar /opt/hadoop-3.2.1/share/hadoop/tools/lib/hadoop-streaming-3.2.1.jar -file ./mapper.py -file ./reducer.py -mapper mapper.py -reducer reducer.py -input ./TP/data/ventes.txt -output output-data1

hadoop fs -cat ./output-data1/part-00000

- Quelle est la totalité des ventes du magasin de Buffalo ?

In [ ]:
483.82

- Créez un nouveau mapper/reducer pour trouver le total des ventes par catégorie de produits.

In [ ]:
#mapper2.py


In [ ]:
#Le reducer NE CHANGE PAS!

- Exécutez un job hadoop sur le fichier ventes.txt en utilisant les fichiers mapper2.py et reducer.py 

- Quelle est la valeur des ventes pour la catégorie Toys?


- Et pour la catégorie Consumer Electronics?

##### Bonus

- Créez un nouveau mapper/reducer pour faire la liste des montants de ventes pour chaque magasin.


In [ ]:
#mapper3.py


In [ ]:
#Reducer2.py


- Quelle est la valeur de la vente la plus élévée pour les magasins suivants:

o Reno

o Toledo

o Chandler


- Quel est le nombre total des ventes et la valeur totale des ventes de tous magasins confondus?